In [113]:
# pip install gspread-dataframe

In [135]:
import pandas as pd
import requests
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.parse import quote
from datetime import datetime, timedelta
import pytz


In [138]:
# Current time and 24 hours ago
today_date = datetime.now().strftime('%Y-%m-%d')

# Manually add the Zoho-required timestamps
TIME_IN = f"{today_date}T00:00:00+05:30"
TIME_OUT = f"{today_date}T23:59:59+05:30"

criteria = "(Created_Time:between:" + TIME_IN + "," + TIME_OUT + ")"
params = {"criteria": criteria}


## GENERATE AUTH AND REFRESH CODE

In [79]:
# # Generate Fresh Token
# auth_code = "1000.ea3db0812b8fd2ed3f6f55ab71b71735.59698dafdedb0ca672cf2cc4ef5beeed"
# redirect_uri = "https://your-registered-redirect-url.com"

# post_url = "https://accounts.zoho.in/oauth/v2/token"

# # Use these exact key names
# token_data = {
#     "grant_type": "authorization_code",
#     "client_id": client_id,
#     "client_secret": client_secret,
#     "redirect_uri": redirect_uri,
#     "code": auth_code
# }

# # IMPORTANT: Use data=, not json=
# token_response = requests.post(post_url, data=token_data)

# print(token_response.status_code)
# print(token_response.json())

200
{'error': 'invalid_code'}


In [44]:
access_token = "1000.b195209a9be1175975ce67b0c470758a.59b7a87f1ec1d0aab5cae00a646a112b"
refresh_token = '1000.4144f1c17033fba3fea985e5050e1c2c.c3ced991667aad6fa884b424b7b7360e'
scope = 'ZohoCRM.modules.Leads.READ ZohoCRM.modules.Events.READ ZohoCRM.modules.Deals.READ'

In [128]:
# Refresh Token
refresh_url = "https://accounts.zoho.in/oauth/v2/token"

data = {
    'refresh_token': refresh_token,
    'client_id': client_id,
    'client_secret': client_secret,
    'grant_type': 'refresh_token'
}

response = requests.post(refresh_url, data=data)
new_tokens = response.json()

access_token = new_tokens.get('access_token')

In [129]:
access_token

'1000.53f299b806eaa42c73683ab7684262bd.918b46b6e8ead2c90fd86317d6918d3b'

## FETCH ZOHO LEADS DATA

In [139]:
get_url_leads = "https://www.zohoapis.in/crm/v8/Leads/search"

headers = {"Authorization": f"Zoho-oauthtoken {access_token}"}

get_response = requests.get(get_url_leads, headers=headers, params=params)

print(f"Status Code: {get_response.status_code}")


# try:
# print(get_response.json())
# except Exception:
    # print("Raw Response Text:")
    # print(get_response.text)

raw_records = get_response.json().get('data', [])
df_leads_zoho = pd.json_normalize(raw_records)
df_leads_zoho

Status Code: 200


,Company,GSTIN,Email,$currency_symbol,$field_states,$sharing_permission,Street_2,Last_Activity_Time,Industry,Unsubscribed_Mode,...,Created_By.email,$review_process.approve,$review_process.reject,$review_process.resubmit,$layout_id.display_label,$layout_id.name,$layout_id.id,Modified_By.name,Modified_By.id,Modified_By.email
0,JMD ENTERPRISES,None,None,₹,None,full_access,None,2026-02-16T17:08:53+05:30,None,None,...,vikas.b@sanmarenterprises.com,False,False,False,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com


## FETCH ZOHO DEALS DATA

In [140]:

get_url_deals = "https://www.zohoapis.in/crm/v8/Deals/search"

headers = {"Authorization": f"Zoho-oauthtoken {access_token}"}

get_response = requests.get(get_url_deals, headers=headers, params=params)

print(f"Status Code: {get_response.status_code}")


# try:
# print(get_response.json())
# except Exception:
    # print("Raw Response Text:")
    # print(get_response.text)

raw_records = get_response.json().get('data', [])
df_deals_zoho = pd.json_normalize(raw_records)
df_deals_zoho




Status Code: 200


,$currency_symbol,$field_states,$sharing_permission,Last_Activity_Time,$process_flow,Deal_Name,Stage,$locked_for_me,id,Created_Time,...,$layout_id.display_label,$layout_id.name,$layout_id.id,Modified_By.name,Modified_By.id,Modified_By.email,Account_Name.name,Account_Name.id,Contact_Name.name,Contact_Name.id
0,₹,None,full_access,2026-02-16T17:31:11+05:30,False,Sania Demo / Sandeep Marwaha,Share Proposal,False,995462000005057369,2026-02-16T17:10:38+05:30,...,Standard,Standard,995462000000507307,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com,Sania Demo,995462000005057363,Sandeep Marwaha,995462000005057366
1,₹,None,full_access,2026-02-16T12:56:06+05:30,False,Blue Craft Agro Pvt Ltd / Sanjeev Kakkar,Receive Requirement Mail,False,995462000005048400,2026-02-16T12:42:04+05:30,...,Standard,Standard,995462000000507307,Vikas Bhambi,995462000000816001,vikas.b@sanmarenterprises.com,Blue Craft Agro Pvt Ltd,995462000005048394,Sanjeev Kakkar,995462000005048397
2,₹,None,full_access,2026-02-16T11:03:45+05:30,False,DELVAL MAKE FULL BORE BALL VALVE SIZE 1.5 INCH...,Share Proposal,False,995462000005057055,2026-02-16T11:02:54+05:30,...,Standard,Standard,995462000000507307,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com,CHANDIGARH DISTILLERS & BOTTLERS LTD,995462000000912001,Vivek Dwedi,995462000004977049


## SETUP GOOGLE SHEETS

In [56]:

# # 1. Authenticate using User Credentials (Client ID & Secret)
# gc = gspread.oauth(
#     credentials_filename='credentials.json',
#     authorized_user_filename='authorized_user.json'  # Token is saved here automatically
# )

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=44783578157-ja6rq4b2bml338k603i5euv6rlva2g42.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60388%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=8Y1S6NNtGqvlu4gpxix1IxjJ4uYd9I&access_type=offline


In [109]:
sheet_url = "https://docs.google.com/spreadsheets/d/11_7I0Juswl_XWysMBL5CW2VXZ235AOxMGm-kEVtQPI4/edit?gid=0#gid=0"

file_name = gc.open_by_url(sheet_url)

worksheet_leads = sh.worksheet("Leads")
worksheet_deals = sh.worksheet("Deals")

# data_leads = worksheet_leads.get_all_records()
# data_deals = worksheet_deals.get_all_records()



In [114]:
# set_with_dataframe(worksheet_leads, df_leads_zoho, row=1, col=1, include_index=False, include_column_header=True, resize=True)
# set_with_dataframe(worksheet_deals, df_deals_zoho, row=1, col=1, include_index=False, include_column_header=True, resize=True)

In [108]:
# Convert DF to list of lists; use .astype(str) to avoid JSON errors with dates
leads_data_to_append = df_leads_zoho.astype(str).values.tolist()
deals_data_to_append = df_deals_zoho.astype(str).values.tolist()

# 3. Append
worksheet_leads.append_rows(leads_data_to_append, value_input_option='USER_ENTERED')
worksheet_deals.append_rows(deals_data_to_append, value_input_option='USER_ENTERED')


print("Data appended successfully!")

APIError: APIError: [400]: Invalid value at 'data.values[0]' (type.googleapis.com/google.protobuf.ListValue), "$currency_symbol"
Invalid value at 'data.values[1]' (type.googleapis.com/google.protobuf.ListValue), "$field_states"
Invalid value at 'data.values[2]' (type.googleapis.com/google.protobuf.ListValue), "$sharing_permission"
Invalid value at 'data.values[3]' (type.googleapis.com/google.protobuf.ListValue), "Last_Activity_Time"
Invalid value at 'data.values[4]' (type.googleapis.com/google.protobuf.ListValue), "$process_flow"
Invalid value at 'data.values[5]' (type.googleapis.com/google.protobuf.ListValue), "Deal_Name"
Invalid value at 'data.values[6]' (type.googleapis.com/google.protobuf.ListValue), "Stage"
Invalid value at 'data.values[7]' (type.googleapis.com/google.protobuf.ListValue), "$locked_for_me"
Invalid value at 'data.values[8]' (type.googleapis.com/google.protobuf.ListValue), "id"
Invalid value at 'data.values[9]' (type.googleapis.com/google.protobuf.ListValue), "Create

In [145]:
data_leads = pd.DataFrame(worksheet_leads.get_all_records())
data_deals = pd.DataFrame(worksheet_deals.get_all_records())

In [147]:
# 1. Ensure the date column is in datetime format
data_leads['Created_Time'] = pd.to_datetime(data_leads['Created_Time'])

# 2. Sort by the ID and Date (ascending)
# This puts the newest record at the bottom for each ID
data_leads = data_leads.sort_values(['id', 'Created_Time'])

# 3. Drop duplicates, keeping only the last (newest) occurrence
data_leads = data_leads.drop_duplicates(subset='id', keep='last')

In [148]:
data_leads

,Company,GSTIN,Email,$currency_symbol,$field_states,$sharing_permission,Street_2,Last_Activity_Time,Industry,Unsubscribed_Mode,...,Created_By.email,$review_process.approve,$review_process.reject,$review_process.resubmit,$layout_id.display_label,$layout_id.name,$layout_id.id,Modified_By.name,Modified_By.id,Modified_By.email
56,VRV HOSPITALITY,,,₹,,full_access,,2025-11-07T15:36:05+05:30,,,...,vikas.b@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
57,Varun beverages Damtal,,,₹,,full_access,,2025-11-07T15:35:52+05:30,,,...,vikas.b@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
54,"Piccadilly agro, BHADSON SUGAR MILL",,,₹,,full_access,,2025-11-10T16:29:56+05:30,,,...,vikas.b@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
55,Nakshatra Biofuels,,,₹,,full_access,,2025-11-11T10:32:29+05:30,,,...,vikas.b@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
52,Adikari Pvt ltd,,,₹,,full_access,,2025-11-12T11:08:05+05:30,,,...,aditya@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
53,Devan Morden,,,₹,,full_access,,2025-11-12T11:07:52+05:30,,,...,aditya@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
48,Sud pine,,,₹,,full_access,,2025-11-17T10:13:24+05:30,,,...,aditya@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Aditya Pagotra,995462000000832001,aditya@sanmarenterprises.com
51,Sudarshan Rolling Mill,,,₹,,full_access,,2025-11-12T20:29:27+05:30,,,...,aditya@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Aditya Pagotra,995462000000832001,aditya@sanmarenterprises.com
49,Uflex unit 2,,,₹,,full_access,,2025-11-14T12:48:50+05:30,Others,,...,aditya@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com
50,Jai Beverage Unit 1,,,₹,,full_access,,2025-11-14T12:48:15+05:30,Food & Beverages,,...,aditya@sanmarenterprises.com,FALSE,FALSE,FALSE,Standard,Standard,995462000000507321,Sanmar Enterprises,995462000000410001,sales@sanmarenterprises.com


In [150]:
client_secret

'a60b2ee703b497635327e97999cd563f2c2d436a0e'